In [ ]:
# Importing Libraries
!pip install tensorflow-gpu==2.1.0-rc1
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

!pip install tqdm
from tqdm import tqdm_notebook

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import subprocess
import zipfile

In [ ]:
# Importing the zip file containing images and extracting it to local directory
dataset_path = "./drive/My Drive/dataset2.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

In [ ]:
# Adding path for later reference when preprocessing images of training and test dataset
dataset_path_new = "./dataset2/"
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [ ]:
# Importing pre-trained model
IMG_SHAPE = (128, 128, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [ ]:
# Freezing weights of pre-trained model
base_model.trainable = False

In [ ]:
# Adding global_average_layer and output layer on top of the imported model
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
prediction_layer = tf.keras.layers.Dense(units=4, activation='softmax')(global_average_layer)

In [ ]:
# Defining the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)


In [ ]:
# Compiling the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [ ]:
# Normalizing images
data_gen_train = ImageDataGenerator(rescale=1/255., zoom_range=0.2, horizontal_flip=True)
data_gen_valid = ImageDataGenerator(rescale=1/255., zoom_range=0.2, horizontal_flip=True)

In [ ]:
# Importing the train dataset from directory
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=10, class_mode="categorical")

Found 4000 images belonging to 4 classes.


In [ ]:
# Importing the test dataset from directory
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=10, class_mode="categorical")

Found 2000 images belonging to 4 classes.


In [ ]:
# Training the model
model.fit_generator(train_generator, epochs=3, validation_data=valid_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 200 steps
Epoch 1/3
400/400 [==============================] - 94s 236ms/step - loss: 1.0317 - categorical_accuracy: 0.5815 - val_loss: 0.7473 - val_categorical_accuracy: 0.6990
Epoch 2/3
400/400 [==============================] - 90s 226ms/step - loss: 0.5099 - categorical_accuracy: 0.8288 - val_loss: 0.5481 - val_categorical_accuracy: 0.7970
Epoch 3/3
400/400 [==============================] - 90s 226ms/step - loss: 0.3834 - categorical_accuracy: 0.8735 - val_loss: 0.4999 - val_categorical_accuracy: 0.8110


In [ ]:
# Transfer learning model evaluation
valid_loss, valid_accuracy = model.evaluate(valid_generator)

  ...
    to  
  ['...']
200/200 [==============================] - 33s 163ms/step - loss: 0.5207 - categorical_accuracy: 0.8055


In [ ]:
# Saving model
savedmodel_dir = "./saved_model/"

tf.keras.models.save_model(
    model,
    savedmodel_dir,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./saved_model/assets


In [ ]:
def prepare(filename):
  img = cv2.imread(filename)
  fix_img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  reshaped_image = cv2.resize(fix_img, (128,128))
  rescaled_image = reshaped_image / 255.
  final_img = np.expand_dims(rescaled_image, axis = 0) 
  return final_img

In [ ]:
import cv2
def process(filename):
  img = cv2.imread(filename)
  fix_img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  final_img = cv2.resize(fix_img, (128,128))
  return final_img

In [ ]:
ty = prepare('X.jpg')
print(type(ty))

<class 'numpy.ndarray'>


In [ ]:
prediction = model.predict([prepare('X.jpg')])
print(prediction)

[[3.1772455e-05 9.9990833e-01 1.6635871e-05 4.3197557e-05]]


In [ ]:
prediction = model.predict([prepare('P1.png')])
print(prediction)

[[2.8142972e-06 1.8109606e-08 7.6563520e-06 9.9998951e-01]]


In [ ]:
prediction = model.predict([prepare('P2.png')])
print(prediction)

[[2.9339037e-07 4.0480550e-08 1.5825850e-09 9.9999964e-01]]


In [ ]:
prediction = model.predict([prepare('F1.jpg')])
print(prediction)

[[4.0148329e-06 4.3282933e-11 9.9793398e-01 2.0619687e-03]]


In [ ]:
prediction = model.predict([prepare('F3.jpg')])
print(prediction)

[[1.8293582e-08 3.9808650e-09 9.9972695e-01 2.7307574e-04]]
